In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [7]:
class DropoutNet(nn.Module):
    def __init__(self):
        super(DropoutNet, self).__init__()
        # data shape : 28*28 = 784
        self.fc1 = nn.Linear(784, 500)
        self.dropout = nn.Dropout(p=0.5) # p=0.5 50%
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self,x):
        x = x.view(x.size(0), -1) # flatten
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [13]:
#model without dropout
class NonDropoutNet(nn.Module):
    def __init__(self):
        super(NonDropoutNet, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self,x):
        x = x.view(x.size(0), -1) # flatten
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [9]:
train_transform = transforms.Compose([
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.3,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.3,))
])

train_data = datasets.MNIST(root="./data/mnist", train=True, download=False,transform=train_transform)
test_data = datasets.MNIST(root="./data/mnist", train=False, download=False,transform=train_transform)

train_loader = torch.utils.data.DataLoader(train_data,batch_size=64,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=64,shuffle=False)


In [10]:
dropout_model = DropoutNet()
print(dropout_model)

dropout_criterion = nn.CrossEntropyLoss()
dropout_optim = optim.SGD(dropout_model.parameters(), lr=0.001)

DropoutNet(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)


In [11]:
for epoch in range(10):
    dropout_model.train()
    for imgs,labels in train_loader:
        dropout_optim.zero_grad()
        dropout_output = dropout_model(imgs)
        dropout_loss = dropout_criterion(dropout_output, labels)
        dropout_loss.backward()
        dropout_optim.step()
        

dropout_model.eval()
with torch.no_grad():
    dropout_correct = 0
    dropout_total = 0
    
    for imgs, labels in test_loader:
        test_output = dropout_model(imgs)
        _, dropout_pre = torch.max(test_output.data, 1)
        dropout_total += imgs.size(0)
        dropout_correct += (dropout_pre == labels).sum().item()
        
    print("dropout ACC >> {:.2f}".format(100 * dropout_correct/dropout_total))
    

dropout ACC >> 82.69


In [14]:
non_dropout_model = NonDropoutNet()

non_dropout_criterion = nn.CrossEntropyLoss()
non_dropout_optim = optim.SGD(non_dropout_model.parameters(), lr=0.001)

for epoch in range(10):
    non_dropout_model.train()
    for imgs, labels in train_loader:
        non_dropout_optim.zero_grad()
        no_output = non_dropout_model(imgs)
        no_loss = non_dropout_criterion(no_output, labels)
        no_loss.backward()
        non_dropout_optim.step()
        
        
        
non_dropout_model.eval()
with torch.no_grad():
    no_correct = 0
    no_total = 0
    
    for imgs, labels in test_loader:
        output = non_dropout_model(imgs)
        _, pred = torch.max(output.data, 1)
        no_total += labels.size(0)
        no_correct += (pred == labels).sum().item()

In [15]:
print("dropout ACC >> {:.2f}".format(100 * no_correct/no_total))

dropout ACC >> 82.60
